In [28]:
DATA_FILE_NAME = "jorf_2023.csv"
VOC_FILE_NAME = "jorf_vocabulary.txt"

# LOAD DATA

In [29]:
import os 
import pandas as pd

directory_path = os.getcwd()
df = pd.read_csv(directory_path + "/" + DATA_FILE_NAME, sep='|', names= ["text_id", "child_id", "type", "article", "paragraph", "text"])
print(df.shape)
df.head()

(454301, 6)


,text_id,child_id,type,article,paragraph,text
0,JORFTEXT000048734585,NaN,0,NaN,0,fr/lr/loi/2023-1380/2023-12-31
1,JORFTEXT000048734585,JORFVERS000048734585,0,NaN,0,LOI n° 2023-1380 du 30 décembre 2023 visant à ...
2,JORFTEXT000048734585,JORFARTI000048734586,1,1,1,I. — Après l'article L. 2122-19 du code généra...
3,JORFTEXT000048734585,JORFARTI000048734586,1,1,2,« Art. L. 2122-19-1. — Pour assurer les foncti...
4,JORFTEXT000048734585,JORFARTI000048734586,1,1,3,II. — L'article L. 2122-19-1 du code général d...


In [30]:
df.loc[3, "text"]

"« Art. L. 2122-19-1. — Pour assurer les fonctions liées au secrétariat de mairie dans les communes de moins de 3 500 habitants, le maire nomme un agent aux fonctions de secrétaire général de mairie, sauf s'il nomme un agent pour occuper les fonctions de directeur général des services. Le secrétaire général de mairie peut exercer ses fonctions à temps partiel ou à temps non complet. »"

# Extract and Persist "text" Column

In [31]:
df_text = df.loc[:,"text"][df["type"] == 1] # Type 0 : law title, Type 1: law description, Type 2: TABLE
print(df_text.shape)
df_text.head()

(406015,)


2    I. — Après l'article L. 2122-19 du code généra...
3    « Art. L. 2122-19-1. — Pour assurer les foncti...
4    II. — L'article L. 2122-19-1 du code général d...
5    « Art. L. 2122-19-1. — Pour assurer les foncti...
6    « Pour assurer les fonctions liées au secrétar...
Name: text, dtype: object

In [32]:
with open(directory_path + "/" + VOC_FILE_NAME, mode="w") as f:
    f.write('\n'.join(df_text.astype(str)))

# Train Tokenizer Model

In [33]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(input=directory_path + "/" + VOC_FILE_NAME, model_prefix='juritok', vocab_size=1000)


sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /home/adrian/Desktop/NLP/juritok/juritok/jorf_vocabulary.txt
  input_format: 
  model_prefix: juritok
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_

# Check Model outputs

In [34]:
sp = spm.SentencePieceProcessor(model_file= directory_path + '/juritok.model')

In [35]:
len(sp)

1000

In [36]:
some_text = df.loc[3, "text"]
print(sp.encode(some_text, out_type=int))

[38, 431, 9, 44, 9, 503, 51, 574, 113, 18, 58, 9, 4, 45, 389, 4, 853, 22, 251, 3, 10, 36, 178, 24, 170, 21, 135, 15, 502, 67, 5, 386, 95, 8, 61, 22, 405, 26, 5, 542, 5, 181, 196, 341, 228, 245, 75, 41, 3, 7, 23, 121, 134, 495, 158, 82, 4, 204, 65, 83, 251, 3, 5, 852, 302, 5, 386, 95, 8, 7, 268, 32, 87, 4, 3, 6, 94, 495, 158, 82, 4, 204, 65, 64, 4, 225, 21, 32, 39, 27, 22, 251, 3, 5, 324, 302, 17, 255, 9, 89, 852, 302, 5, 386, 95, 8, 333, 4, 707, 20, 388, 251, 3, 16, 838, 541, 37, 47, 16, 838, 504, 192, 66, 15, 9, 73]


In [37]:
print(sp.encode(some_text, out_type=str))

['▁«', '▁Art', '.', '▁L', '.', '▁21', '2', '2-1', '9', '-', '1', '.', '▁', '—', '▁Pour', '▁', 'assurer', '▁les', '▁fonction', 's', '▁l', 'i', 'ées', '▁au', '▁se', 'c', 'ré', 't', 'ari', 'at', '▁de', '▁mai', 'ri', 'e', '▁dans', '▁les', '▁commun', 'es', '▁de', '▁moins', '▁de', '▁3', '▁5', '00', '▁h', 'ab', 'it', 'ant', 's', ',', '▁le', '▁m', 'aire', '▁nom', 'me', '▁un', '▁', 'ag', 'ent', '▁aux', '▁fonction', 's', '▁de', '▁secrétaire', '▁général', '▁de', '▁mai', 'ri', 'e', ',', '▁sa', 'u', 'f', '▁', 's', "'", 'il', '▁nom', 'me', '▁un', '▁', 'ag', 'ent', '▁pour', '▁', 'oc', 'c', 'u', 'p', 'er', '▁les', '▁fonction', 's', '▁de', '▁directeur', '▁général', '▁des', '▁services', '.', '▁Le', '▁secrétaire', '▁général', '▁de', '▁mai', 'ri', 'e', '▁peut', '▁', 'exerce', 'r', '▁ses', '▁fonction', 's', '▁à', '▁temps', '▁partie', 'l', '▁ou', '▁à', '▁temps', '▁non', '▁comp', 'le', 't', '.', '▁»']


In [38]:
print(sp.encode_as_pieces([some_text]))

[['▁«', '▁Art', '.', '▁L', '.', '▁21', '2', '2-1', '9', '-', '1', '.', '▁', '—', '▁Pour', '▁', 'assurer', '▁les', '▁fonction', 's', '▁l', 'i', 'ées', '▁au', '▁se', 'c', 'ré', 't', 'ari', 'at', '▁de', '▁mai', 'ri', 'e', '▁dans', '▁les', '▁commun', 'es', '▁de', '▁moins', '▁de', '▁3', '▁5', '00', '▁h', 'ab', 'it', 'ant', 's', ',', '▁le', '▁m', 'aire', '▁nom', 'me', '▁un', '▁', 'ag', 'ent', '▁aux', '▁fonction', 's', '▁de', '▁secrétaire', '▁général', '▁de', '▁mai', 'ri', 'e', ',', '▁sa', 'u', 'f', '▁', 's', "'", 'il', '▁nom', 'me', '▁un', '▁', 'ag', 'ent', '▁pour', '▁', 'oc', 'c', 'u', 'p', 'er', '▁les', '▁fonction', 's', '▁de', '▁directeur', '▁général', '▁des', '▁services', '.', '▁Le', '▁secrétaire', '▁général', '▁de', '▁mai', 'ri', 'e', '▁peut', '▁', 'exerce', 'r', '▁ses', '▁fonction', 's', '▁à', '▁temps', '▁partie', 'l', '▁ou', '▁à', '▁temps', '▁non', '▁comp', 'le', 't', '.', '▁»']]


In [39]:
print(sp.decode([['▁«', '▁Art', '.', '▁L', '.', '▁', '21', '22', '-', '1', '9', '-', '1', '.', '▁—', '▁Pour', '▁', 'ass', 'ure', 'r', '▁les', '▁fonction', 's', '▁l', 'i', 'é', 'es', '▁au', '▁se', 'c', 'ré', 't', 'ar', 'i', 'at', '▁de', '▁m', 'a', 'ir', 'ie', '▁dans', '▁les', '▁comm', 'une', 's', '▁de', '▁m', 'o', 'in', 's', '▁de', '▁3', '▁5', '0', '0', '▁h', 'ab', 'it', 'ant', 's', ',', '▁le', '▁m', 'aire', '▁n', 'om', 'me', '▁un', '▁', 'ag', 'ent', '▁aux', '▁fonction', 's', '▁de', '▁se', 'c', 'ré', 't', 'aire', '▁général', '▁de', '▁m', 'a', 'ir', 'ie', ',', '▁s', 'a', 'u', 'f', '▁s', "'", 'il', '▁n', 'om', 'me', '▁un', '▁', 'ag', 'ent', '▁pour', '▁', 'oc', 'c', 'u', 'p', 'er', '▁les', '▁fonction', 's', '▁de', '▁directeur', '▁général', '▁des', '▁service', 's', '.', '▁Le', '▁se', 'c', 'ré', 't', 'aire', '▁général', '▁de', '▁m', 'a', 'ir', 'ie', '▁peut', '▁ex', 'er', 'c', 'er', '▁se', 's', '▁fonction', 's', '▁à', '▁', 't', 'emp', 's', '▁par', 'ti', 'el', '▁ou', '▁à', '▁', 't', 'emp', 's', '▁n', 'on', '▁comp', 'le', 't', '.', '▁»']]))

["« Art. L. 2122-19-1.▁— Pour assurer les fonctions liées au secrétariat de mairie dans les▁communes de moins de 3 500 habitants, le maire nomme un agent aux fonctions de secrétaire général de mairie,▁sauf▁s'il nomme un agent pour occuper les fonctions de directeur général des services. Le secrétaire général de mairie peut exercer ses fonctions à temps partiel ou à temps non complet. »"]
